# Count the Number of Powerful Integers

You are given three integers start, finish, and limit. You are also given a 0-indexed string s representing a positive integer.

A positive integer x is called powerful if it ends with s (in other words, s is a suffix of x) and each digit in x is at most limit.

Return the total number of powerful integers in the range [start..finish].

A string x is a suffix of a string y if and only if x is a substring of y that starts from some index (including 0) in y and extends to the index y.length - 1. For example, 25 is a suffix of 5125 whereas 512 is not. 

In [ ]:

class Solution:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        low = str(start)
        high = str(finish)
        n = len(high)
        low = low.zfill(n)  # 자릿수를 맞추기 위해 왼쪽에 0 채우기
        pre_len = n - len(s)  # 접두사 길이 계산
        
        @cache
        def dfs(i, limit_low, limit_high):
            # 재귀 종료 조건: 모든 자릿수를 처리했을 때
            if i == n:
                return 1
                
            # 현재 자릿수에서 사용할 수 있는 숫자의 범위 계산
            lo = int(low[i]) if limit_low else 0  # 하한 제약이 있으면 low[i], 없으면 0
            hi = int(high[i]) if limit_high else 9  # 상한 제약이 있으면 high[i], 없으면 9
            res = 0
            
            if i < pre_len:
                # 접두사 부분: 0부터 limit까지 숫자 중 가능한 것들 시도
                for digit in range(lo, min(hi, limit) + 1):
                    res += dfs(
                        i + 1,
                        limit_low and digit == lo,  # 현재 숫자가 하한과 같으면 다음 자리도 하한 제약 유지
                        limit_high and digit == hi,  # 현재 숫자가 상한과 같으면 다음 자리도 상한 제약 유지
                    )
            else:
                # 접미사 부분: s의 해당 자리 숫자만 사용 가능
                x = int(s[i - pre_len])  # s의 현재 위치 숫자
                if lo <= x <= min(hi, limit):  # s의 숫자가 범위 내에 있는지 확인
                    res = dfs(
                        i + 1, 
                        limit_low and x == lo,  # 현재 숫자가 하한과 같으면 다음 자리도 하한 제약 유지
                        limit_high and x == hi,  # 현재 숫자가 상한과 같으면 다음 자리도 상한 제약 유지
                    )
            
            return res
        
        return dfs(0, True, True)  # 처음에는 상한과 하한 모두 제약 있음